## Maximum roductivity and yield 

In [2]:
from cobra.io import read_sbml_model
import os

In [7]:
# Verify working directory and change it if needed
os.getcwd()
os.chdir("/Users/abril/Documents/DTU/CFD_Resveratrol/27410-group-assigment-group-4-resveratrol-in-s-cerevisiae")

In [8]:
model = read_sbml_model("data/models/yeast_resv.xml")

In [10]:
# Model olverview
model

Name,iMM904
Memory address,0x0105bb0df0
Number of metabolites,1229
Number of reactions,1581
Number of groups,0
Objective expression,1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090
Compartments,"cytosol, extracellular space, mitochondria, peroxisome/glyoxysome, endoplasmic reticulum, vacuole, golgi apparatus, nucleus"


### Calculate theoretical maximum yields for chosen product for suitable carbon sources

In [13]:
# Verify if there is already an objective function (model exploration)
print(model.objective)

Maximize
1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090


In [14]:
# The present objective function belongs to:
model.reactions.BIOMASS_SC5_notrace

Reaction identifier,BIOMASS_SC5_notrace
Name,Biomass SC5 notrace
Memory address,0x119943880
Stoichiometry,1.1348 13BDglcn_c + 0.4588 ala__L_c + 0.046 amp_c + 0.1607 arg__L_c + 0.1017 asn__L_c + 0.2975 asp__L_c + 59.276 atp_c + 0.0447 cmp_c + 0.0066 cys__L_c + 0.0036 damp_c + 0.0024 dcmp_c + 0.0024... 1.1348 1 3 beta D Glucan C6H10O5 + 0.4588 L-Alanine + 0.046 AMP C10H12N5O7P + 0.1607 L-Arginine + 0.1017 L-Asparagine + 0.2975 L-Aspartate + 59.276 ATP C10H12N5O13P3 + 0.0447 CMP C9H12N3O8P +...
GPR,
Lower bound,0.0
Upper bound,999999.0


In [15]:
model.reactions.get_by_id('VVVST1')

Reaction identifier,VVVST1
Name,
Memory address,0x1199e9900
Stoichiometry,e4_coumaroyl_coa + 3.0 h_c + 3.0 malcoa_c --> 4.0 co2_c + 4.0 coa_c + trans_resv + 3.0 H+ + 3.0 Malonyl CoA C24H33N7O19P3S --> 4.0 CO2 CO2 + 4.0 Coenzyme A +
GPR,
Lower bound,0.0
Upper bound,999999.0


##### Calculating maximum productivity and theoretical yield in mmol.

In [45]:
## Setting the objective reaction to resveratrol production
with model:
    model.objective = model.reactions.VVVST1
    resveratrol_max_production = model.optimize().objective_value
    print("Maximum resveatrol productivity = ",resveratrol_max_production , "mmol/gDW*h")

    glucose_uptake_flux = model.reactions.EX_glc__D_e.flux #Glucose exchange reaction
    max_yield = resveratrol_max_production / (-1*glucose_uptake_flux)
    print("Maximum theoretical yield is = ", max_yield, "mmol/gDW*h")




Maximum resveatrol productivity =  1.178571428571428 mmol/gDW*h
Maximum theoretical yield is =  0.1178571428571428 mmol/gDW*h


##### Calculating maximum productivity and theoretical yield in C-mole

In [47]:
##  Now lets convert mol to C-mol
# Conversion factors
C=12
N=14
H=1
O=16

resveatrol = C*14+H*12+O*3 #C14H12O3
glucose = 6

# Calculation 
resv_max_prod_cmol = (resveratrol_max_production)/(resveatrol)
resv_max_yield_cmol = (max_yield*resveatrol)/(glucose)
print('Maximum productivity of lycopene =', resv_max_prod_cmol, 'cmol-resv/gDW*h')
print('Maximum theoretical yield =', resv_max_yield_cmol, 'cmol-resv/cmol-glucose')

Maximum productivity of lycopene = 0.005169172932330824 cmol-resv/gDW*h
Maximum theoretical yield = 4.478571428571427 cmol-resv/cmol-glucose


The results show a very low productivity and yield for resveratrol in our model.
* Maximum productivity = 1.178 mmol/gDW*h 
* Maximum theoretical yield = 0.117 mmol/gDW*h 

* Maximum productivity =  0.005 cmol-resv/gDW*h
* Maximum theoretical yield = 4.478 cmol-resv/cmol-glucose

##### Now it is time to increase the productivity and yiend in the model.